## Install ```openai-agents``` SDK

In [1]:
pip install openai-agents

Note: you may need to restart the kernel to use updated packages.


d:\My Learning\GIAIC\agents-sdk-crash-course\.venv\Scripts\python.exe: No module named pip


## Make the notebook capable of running the asynchronous functions

In [2]:
import nest_asyncio
nest_asyncio.apply()

# Run Google Gemini with OpenAI-Agents SDK

In [3]:
import os
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel, RunConfig

In [5]:

gemini_api_key = os.environ.get("GOOGLE_API_KEY")
if not gemini_api_key:
    raise ValueError("Please set the GOOGLE_API_KEY environment variable with your Google Gemini API key.")

externalClient = AsyncOpenAI(
    api_key = gemini_api_key,
    base_url = "https://generativelanguage.googleapis.com/v1beta/openai", 
)

model = OpenAIChatCompletionsModel(
    openai_client = externalClient,
    model = "gemini-2.5-flash",
)

# Run level configuration
config = RunConfig(
    model = model,
    model_provider=externalClient,
    tracing_disabled=True
)


In [10]:
import asyncio
from agents import Agent, Runner, RunConfig
from openai.types.responses import ResponseTextDeltaEvent

## Streaming Text Code

In [13]:

async def main():
    agent = Agent(
        name = "joker agent",
        instructions = "You are a famous joker in Pakistani community.",
        model = model
    )

    # asynchronous streamed run
    result = Runner.run_streamed(agent,"tell me 5 jokes on PTI", run_config=config)

    async for event in result.stream_events():
        # print(chunk)
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            print(event.data.delta, end="", flush=True)

asyncio.run(main())


Salam alaikum, mere pyare bhaiyo aur behno! Kya haal hain? Aapka apna joker haazir hai, aur aaj mood hai thora sa siyasat pe hansi mazaaq karne ka. Lekin yaad rakhiye, sirf mazaaq hai, dil pe na lena, theek hai? Hum sab aik hi mulk ke log hain, hansi khushi raho!

Chalo phir, shuru karte hain PTI pe 5 masalaydaar jokes:

1.  **U-Turn Special:**
    Aik dafa aik banda Captain se directions pooch raha tha, "Khan Sahib, airport ka rasta kidhar hai?" Captain sahib ne aik rasta bataya, phir agle hi pal kaha, "Nahi, nahi! U-turn le lo, woh pehla wala rasta theek nahi tha!" Banda bola, "Haye Allah! Kya aapka GPS bhi PTI ne banaya hai?!"

2.  **Naya Pakistan Ki Mehngai:**
    Aik dost apne doosre dost se pooch raha tha, "Yaar, ye Naya Pakistan mein sab kuch itna mehenga kyun ho gaya hai? Murgi bhi BMW mein ghoomti lagti hai!" Doosra dost hans ke bola, "Bhai, Naya Pakistan ka slogan hi hai 'Ghabrana Nahin', lekin ab to dukandar bhi cheezain bechte hue ghabra jate hain ke itni mehengi hain!"

3.

## Streaming item code

In [14]:
import asyncio
import random

from agents import Agent, ItemHelpers, Runner, function_tool


@function_tool
def how_many_jokes() -> int:
    return random.randint(1, 10)


async def main():
    agent = Agent(
        name="Joker",
        instructions="First call the `how_many_jokes` tool, then tell that many jokes.",
        tools=[how_many_jokes],
        model=model
    )

    result = Runner.run_streamed(
        agent,
        input="Hello",

    )
    print("=== Run starting ===")
    async for event in result.stream_events():
        # We'll ignore the raw responses event deltas
        if event.type == "raw_response_event":
            continue
        elif event.type == "agent_updated_stream_event":
            print(f"Agent updated: {event.new_agent.name}")
            continue
        elif event.type == "run_item_stream_event":
            if event.item.type == "tool_call_item":
                print("-- Tool was called")
            elif event.item.type == "tool_call_output_item":
                print(f"-- Tool output: {event.item.output}")
            elif event.item.type == "message_output_item":
                print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
            else:
                pass  # Ignore other event types





asyncio.run(main())

print("=== Run complete ===")

=== Run starting ===
Agent updated: Joker
-- Tool was called
-- Tool output: 2
-- Message output:
 Why did the scarecrow win an award? Because he was outstanding in his field!

What do you call a fake noodle? An impasta!
=== Run complete ===
